In [1]:
from sklearn.externals import joblib
import os,sys

In [2]:
def CheckMeCabisImported():
    importDetected = 0
    for index in sys.path:
        if index == '/usr/lib/python2.7':
            print 'Imported'
            importDetected = 1
            break
    if importDetected == 0:
        print 'Not Imported'
        ImportMeCab()
def ImportMeCab():
    sys.path.append('/usr/lib/python2.7')
    sys.path.append('/usr/lib/python2.7/plat-x86_64-linux-gnu')
    sys.path.append('/usr/local/lib/python2.7/dist-packages')
    sys.path.append('/usr/lib/python2.7/dist-packages')
    sys.path.append('/usr/lib/pymodules/python2.7')
    print 'Now Imported'

In [3]:
def DeleteSpecialCharacter(target):
    specialCharacter = ".#/?:${}}[]()*&^%@~_-+=!,><;"
    for character in specialCharacter:
        target = target.replace(character, ' ')
    return target

In [4]:
def AnyGramForAscii(target, characterGap):
    strLength = len(target)
    if characterGap >= int(strLength):
        return target.replace(' ', '')
    elif characterGap == 0:
        return target
    else:
        resultStr = ""
        #target = list(target)
        for eachCharacterStartPoint in range(len(target) - characterGap + 1):
            for eachCharacterSavePoint in range(characterGap):
                #print eachCharacterSavePoint
                resultStr = resultStr + target[eachCharacterStartPoint +
                                              eachCharacterSavePoint]
            resultStr = resultStr + ' '
        return resultStr

In [5]:
def AnyGram(target, characterGap):
    strLength = len(target)
    if characterGap >= int(strLength/3):
        return target.replace(' ', '')
    elif characterGap == 0:
        return target
    else:
        resultStr = ""
        target = list(target)
        for eachCharacterStartMemoryPoint in range(int(len(target)/3) - characterGap + 1):
            strTemp = ""
            for saveCharacterPoint in range(characterGap * 3):
                strTemp = strTemp + target[
                    eachCharacterStartMemoryPoint * 3 + saveCharacterPoint]
            resultStr = resultStr + strTemp + ' '
        return resultStr

In [6]:
def ParseKoreanWithMeCab(target):
    mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')
    return mecab.parse(target)
def ParseMeCabResult(target):
    writeAvailable = 1
    resultStr = ""
    for character in target:
        if writeAvailable == 1 and character != '\t':
            resultStr = resultStr + character
        if character == '\t':
            writeAvailable = 0
            resultStr = resultStr + ' '
        elif character == '\n':
            writeAvailable = 1
    
    resultStr.replace('EOS','')
    resultStr = resultStr[:-5]
    return resultStr

In [7]:
CheckMeCabisImported()
import MeCab

Not Imported
Now Imported


In [8]:

clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')


In [9]:
joblib.dump(clf,'n_classify.model')

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [10]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat']

In [11]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [ ]:
testCase = u'[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)'
testCase = DeleteSpecialCharacter(testCase)
parseCase = testCase.encode('utf-8')
parseCase = ParseKoreanWithMeCab(parseCase)
parseCase = ParseMeCabResult(parseCase)
print parseCase
#testTempCase = testCase
#testTempCase = AnyGram(testCase, len(testCase))
#print testTempCase
#for index in range(1, 3):
#    testCase = testCase + ' ' + AnyGram(testTempCase, index)
#print testCase
#pred = clf.predict(vectorizer.transform([testCase]))[0]
#print cate_id_name_dict[pred]

신한 카드 5 할인 서우 한복 아동 한복 여자 아동 금 나래 분홍


In [ ]:
from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    #print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    name = DeleteSpecialCharacter(name)
    name = name.decode('utf-8').encode('utf-8')
    #print name
    parseCase = ParseKoreanWithMeCab(name)
    parseCase = ParseMeCabResult(parseCase)
    name = name + ' ' + parseCase
    print name
    pred = clf.predict(vectorizer.transform([name]))[0]
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)


Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



 신한카드5 할인  예화 좋은아이들  아동한복 여아 1076 빛이나노랑 신한 카드 5 할인 예 화 좋 은 아이 들 아동 한복 여아 1076 빛 이나 노랑


127.0.0.1 - - [23/Aug/2016 07:56:59] "GET /classify?name=[%EC%8B%A0%ED%95%9C%EC%B9%B4%EB%93%9C5%%ED%95%A0%EC%9D%B8][%EC%98%88%ED%99%94-%EC%A2%8B%EC%9D%80%EC%95%84%EC%9D%B4%EB%93%A4]%20%EC%95%84%EB%8F%99%ED%95%9C%EB%B3%B5%20%EC%97%AC%EC%95%84%201076%20%EB%B9%9B%EC%9D%B4%EB%82%98%EB%85%B8%EB%9E%91&img= HTTP/1.1" 200 74


인텔CPU코어 i3 4세대 하스웰 리프레시 4160 3 6GHz INTEL  인텔 CPU 코어 i 3 4 세대 하스 웰 리프 레시 4160 3 6 GHz INTEL
 4 즉시할인쿠폰 인텔 intel 코어4세대 하스웰 i3 4160 4 즉시 할인 쿠폰 인텔 intel 코어 4 세대 하스 웰 i 3 4160
 해외  구매대행  B00MMVIFZM  Voltage References 3 3V Prec Micropwr LDO Low VRef  5 pieces  해외 구매 대행 B 00 MMVIFZM Voltage References 3 3 V Prec Micropwr LDO Low VRef 5 pieces
 해외  구매대행  B00AZVJIEI  AD7524LNZ 해외 구매 대행 B 00 AZVJIEI AD 7524 LNZ
 정품  니콘 DSLR D800E Body 맨프로토 MKBFRA4 BH 삼각대  인화권 웹하드 정품 니콘 DSLR D 800 E Body 맨 프로 토 MKBFRA 4 BH 삼각대 인화 권 웹 하드
 캐논코리아정품  Canon EOS 70D 18 55 IS II STM  3355가방   SDHC 8GB UV필터 리더기 크리너SET 액정필름 융 즐기기책자 그외사은품 캐논 코리아 정품 Canon EOS 70 D 18 55 IS II STM 3355 가방 SDHC 8 GB UV 필터 리더 기 크리너 SET 액정 필름 융 즐기 기 책자 그 외 사은품
자이로컴 자이뷰 XV120PCI BT 4채널 자이로 컴 자이 뷰 XV 120 PCI BT 4 채널
씨앤비텍 BDS1212 씨 앤 비텍 BDS 1212
 D 외장하드 1TB  6 4CM 2 5형 자개에디션 선유도   PCD10184  D 외장 하드 1 TB 6 4 CM 2 5 형 자개 에디션 선유도 PCD 10184


121.140.65.88 - - [23/Aug/2016 07:57:27] "GET /classify?name=%EC%9D%B8%ED%85%94CPU%EC%BD%94%EC%96%B4%20i3-4%EC%84%B8%EB%8C%80%20%ED%95%98%EC%8A%A4%EC%9B%B0%20%EB%A6%AC%ED%94%84%EB%A0%88%EC%8B%9C%204160(3.6GHz/INTEL)&img=http://shopping.phinf.naver.net/main_7975500/7975500771.jpg HTTP/1.1" 200 62
121.140.65.88 - - [23/Aug/2016 07:57:27] "GET /classify?name=[4%%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8%EC%BF%A0%ED%8F%B0]%EC%9D%B8%ED%85%94%20intel%20%EC%BD%94%EC%96%B44%EC%84%B8%EB%8C%80%20%ED%95%98%EC%8A%A4%EC%9B%B0%20i3-4160&img=http://shopping.phinf.naver.net/main_8350292/8350292853.jpg HTTP/1.1" 200 62
121.140.65.88 - - [23/Aug/2016 07:57:27] "GET /classify?name=[%ED%95%B4%EC%99%B8][%EA%B5%AC%EB%A7%A4%EB%8C%80%ED%96%89][B00MMVIFZM]%20Voltage%20References%203.3V%20Prec%20Micropwr%20LDO%20Low%20VRef%20(5%20pieces)&img=http://shopping.phinf.naver.net/main_8266191/8266191804.jpg HTTP/1.1" 200 72
121.140.65.88 - - [23/Aug/2016 07:57:27] "GET /classify?name=[%ED%95%B4%EC%99%B8][%EA%B5%AC%EB%A7%A4%

...

121.140.65.88 - - [23/Aug/2016 07:57:31] "GET /classify?name=[EVISU]%20%EC%97%AC%EC%84%B1_%EB%9D%BC%EC%9D%B4%ED%8A%B8%20%EC%9D%B8%EB%94%94%EA%B3%A0%EC%8A%AC%EB%A6%BC%20%ED%85%8D%EC%8A%A4%ED%8A%B8%ED%94%84%EB%A6%B0%ED%8A%B8%20%EB%8D%B0%EB%8B%98%ED%8C%AC%EC%B8%A0_EN8JP060_BL_P021219638&img=http://shopping.phinf.naver.net/main_8317496/8317496263.jpg HTTP/1.1" 200 86
121.140.65.88 - - [23/Aug/2016 07:57:31] "GET /classify?name=[JEEP]%EB%82%A8%EB%85%80%EA%B3%B5%EC%9A%A9%EC%A0%84%ED%8C%90%ED%94%84%EB%A6%B0%ED%8A%B8%EC%8A%AC%EB%A6%BC%EB%B0%B0%EA%B8%B0%ED%8C%AC%EC%B8%A0_JG1DPF702&img=http://shopping.phinf.naver.net/main_8305062/8305062386.jpg HTTP/1.1" 200 86
121.140.65.88 - - [23/Aug/2016 07:57:31] "GET /classify?name=[%EA%B5%AD%EB%AF%BC%EC%B9%B4%EB%93%9C5%][%EC%A0%9C%EC%9D%B4%EC%A0%9C%EC%9D%B4%EC%98%B4%EB%AF%80(JJHOMME)]%ED%81%B4%EB%9E%98%EC%8B%9D%20%EB%84%A4%EC%9D%B4%EB%B9%84%20%EB%B3%B4%EC%B9%B4%EC%8B%9C%203%EC%A2%85%20%EC%A0%95%EC%9E%A5%EC%88%98%ED%8A%B8%20JJ_15BZ_02SET&img=http://shoppin

 미넴옴므   정장바지 엘모체크포멀정장바지 P1138 PRSSS  미 넴 옴므 정장 바지 엘모 체크 포 멀 정장 바지 P 1138 PRSSS
 오가게 TOKIO 웨이브울니트베스트 그레이  오가 게 TOKIO 웨이브 울 니트 베스트 그레이
 5  즉시할인  오가게   오가게 TOKIO 웨이브울니트베스트그레이  P020278325 5 즉시 할인 오가 게 오가 게 TOKIO 웨이브 울 니트 베스트 그레이 P 020278325
 KB국민카드 5 할인  3 14  디스퀘어드2   QR코드인증 14FW 74LA0651 S30144 470   디퀘 오렌지스팟 핸드커프패치 디스진 슬림핏   중청   남성 청바지 KB 국민카드 5 할인 3 14 디 스퀘어드 2 QR 코드 인증 14 FW 74 LA 0651 S 30144 470 디 퀘 오렌지 스팟 핸드 커 프 패치 디 스진 슬림 핏 중청 남성 청바지
 디스퀘어드2  화이트 페인팅 디스 15SS 슬림   LA0885 S30144 470   남성 진 디 스퀘어드 2 화이트 페인팅 디스 15 SS 슬림 LA 0885 S 30144 470 남성 진
 하프클럽 BOYLONDON  B42TS41U89 하프 클럽 BOYLONDON B 42 TS 41 U 89
 SEOBEAN   서빈 댄디 슬림핏 하프팬츠 SBHNHP 135052 P013862565 SEOBEAN 서빈 댄디 슬림 핏 하프 팬츠 SBHNHP 135052 P 013862565
 엠비오 엠비오 F W 소매 배색 알파카 혼방 네이비 피코트 MK4X30002R  엠 비 오 엠 비오 F W 소매 배색 알파카 혼방 네이비 피코트 MK 4 X 30002 R
 MVIO  소매 배색 알파카 혼방 네이비 피코트  MK4X30002R  MVIO 소매 배색 알파카 혼방 네이비 피코트 MK 4 X 30002 R
 fashionplus  HEAD  소셜 후디드 여성 트레이닝 팬츠 JOHS53221MG fashionplus HEAD 소셜 후디 드 여성 트레이닝 팬츠 JOHS 53221 MG


121.140.65.88 - - [23/Aug/2016 07:57:31] "GET /classify?name=[%EC%97%A0%EB%B9%84%EC%98%A4]%EC%97%A0%EB%B9%84%EC%98%A4%20F/W%20%EC%86%8C%EB%A7%A4%20%EB%B0%B0%EC%83%89%20%EC%95%8C%ED%8C%8C%EC%B9%B4%20%ED%98%BC%EB%B0%A9%20%EB%84%A4%EC%9D%B4%EB%B9%84%20%ED%94%BC%EC%BD%94%ED%8A%B8(MK4X30002R)&img=http://shopping.phinf.naver.net/main_8191868/8191868491.1.jpg HTTP/1.1" 200 74
121.140.65.88 - - [23/Aug/2016 07:57:31] "GET /classify?name=[MVIO]%20%EC%86%8C%EB%A7%A4%20%EB%B0%B0%EC%83%89%20%EC%95%8C%ED%8C%8C%EC%B9%B4%20%ED%98%BC%EB%B0%A9%20%EB%84%A4%EC%9D%B4%EB%B9%84%20%ED%94%BC%EC%BD%94%ED%8A%B8%20(MK4X30002R)&img=http://shopping.phinf.naver.net/main_8054630/8054630582.jpg HTTP/1.1" 200 74
121.140.65.88 - - [23/Aug/2016 07:57:31] "GET /classify?name=[fashionplus][HEAD]%20%EC%86%8C%EC%85%9C%20%ED%9B%84%EB%94%94%EB%93%9C%20%EC%97%AC%EC%84%B1%20%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8B%9D%20%ED%8C%AC%EC%B8%A0_JOHS53221MG&img=http://shopping.phinf.naver.net/main_8329421/8329421460.jpg HTTP/1.1" 200 92
121.1

 BC카드5 할인  HEAD  HEAD  H2X 3LAYER Trail 트레이닝 팬츠 JHHS57011GY 패션플러스  BC 카드 5 할인 HEAD HEAD H 2 X 3 LAYER Trail 트레이닝 팬츠 JHHS 57011 GY 패션 플러스
 MARCELO BURLONE r   마르셀로불론 14SS BM4001 02 10   알라스 퍼플윙   블랙   남성 후드티 P019375800 MARCELO BURLONE r 마르셀 로불 론 14 SS BM 4001 02 10 알라스 퍼플 윙 블랙 남성 후드 티 P 019375800
 하프클럽 HARRISON 러쉬나염 MTM MB1591 하프 클럽 HARRISON 러쉬 나염 MTM MB 1591
 5  즉시할인 Sexy Backless Sleeveless Lace Hip Mini Dress 353 5 즉시 할인 Sexy Backless Sleeveless Lace Hip Mini Dress 353
 5  즉시할인 Rockabilly Deep Puff Bodycon Party Pencil Dress 5 즉시 할인 Rockabilly Deep Puff Bodycon Party Pencil Dress
 블루머핀  플로랄 원피스 BM14FOP606C 블루머 핀 플로 랄 원피스 BM 14 FOP 606 C
 잭키드 주니어  보리보리 잭키드 주니어 델핀조끼정장 여 잭 키드 주니어 보리 보리 잭 키드 주니어 델 핀 조끼 정장 여
 비비안 마름모체크남성트렁크팬티 TP9346  비비안 마름모 체크 남성 트렁크 팬티 TP 9346
 현대백화점 VI관   비비안 마름모체크남성트렁크팬티 TP9346  현대 백화점 VI 관 비비 안 마름모 체크 남성 트렁크 팬티 TP 9346
 BC카드5 할인  예화 좋은아이들  名品 남아 아동한복 옆색동산호  13106  배자   저고리   바지  BC 카드 5 할인 예 화 좋 은 아이 들 名品 남 아 아동 한복 옆 색동 산호 13106 배자 저고리 바지
 KB국민카드 5 할인  3 14 베네통키즈블링팝 

121.140.65.88 - - [23/Aug/2016 07:57:32] "GET /classify?name=[%ED%98%84%EB%8C%80%EB%B0%B1%ED%99%94%EC%A0%90%20VI%EA%B4%80]%20[%EB%B9%84%EB%B9%84%EC%95%88]%EB%A7%88%EB%A6%84%EB%AA%A8%EC%B2%B4%ED%81%AC%EB%82%A8%EC%84%B1%ED%8A%B8%EB%A0%81%ED%81%AC%ED%8C%AC%ED%8B%B0(TP9346)&img=http://shopping.phinf.naver.net/main_8008829/8008829527.jpg HTTP/1.1" 200 123
121.140.65.88 - - [23/Aug/2016 07:57:32] "GET /classify?name=[BC%EC%B9%B4%EB%93%9C5%%ED%95%A0%EC%9D%B8][%EC%98%88%ED%99%94-%EC%A2%8B%EC%9D%80%EC%95%84%EC%9D%B4%EB%93%A4]%20%E5%90%8D%E5%93%81%20%EB%82%A8%EC%95%84%20%EC%95%84%EB%8F%99%ED%95%9C%EB%B3%B5%20%EC%98%86%EC%83%89%EB%8F%99%EC%82%B0%ED%98%B8_%2013106%20(%EB%B0%B0%EC%9E%90%20+%20%EC%A0%80%EA%B3%A0%EB%A6%AC%20+%20%EB%B0%94%EC%A7%80)&img=http://shopping.phinf.naver.net/main_7439535/7439535409.16.jpg HTTP/1.1" 200 74
121.140.65.88 - - [23/Aug/2016 07:57:32] "GET /classify?name=[KB%EA%B5%AD%EB%AF%BC%EC%B9%B4%EB%93%9C%205%%ED%95%A0%EC%9D%B8,%203/14]%EB%B2%A0%EB%84%A4%ED%86%B5%ED%82%A4%EC%A

 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능